In [1]:
import pandas as pd
import numpy as np
import datatable as dt

ModuleNotFoundError: No module named 'datatable'

In [2]:
snp_orig_dt = dt.fread("869_NoCal_CR0.99.tagSNPs_BigLD0.70.txt")
cloud_dens_orig = pd.read_csv("cloud_dens_yearAvg", sep="\t", header=None)
tmin_orig = pd.read_csv("tmin_yearAvg", sep="\t", header=None)

We randomly selected 10,000 features from the dataset to use in our models. Running our models on the full set of features would be computationally infeasible for our machines. 10,000 features was a good number, because we were to able to include a large number of features without the code taking too long to run.    

In [3]:
import random

k = 10000
snp_sample = dt.Frame()
snp_sample.cbind([snp_orig_dt[col] for col in random.sample(snp_orig_dt.names[6:], k)])

snp_sample = snp_sample.to_pandas()

To handle the NAs, rather than throwing out all of the NAs, we apply a KNN classification by taking the 10 closest datapoints in the high dimensional space and taking the most common value. We argue that in doing this rather than taking the mean of the 10 closest data points, we can output a valid value, but more specifically, similar looking variants would have mutated in a similar fashion.

In [22]:
display(snp_sample.iloc[:, 1:10].describe())
display(tmin_orig[2].describe())
display(cloud_dens_orig[2].describe())

,Chr06_23493718_A,Chr01_28502495_T,Chr10_8941925_C,Chr16_1372888_G,Chr16_452183_T,Chr06_20637860_A,Chr01_7473042_G,Chr09_8224355_A,Chr10_8210298_C
count,861.000000,869.000000,869.000000,868.000000,869.000000,868.000000,869.000000,869.000000,869.00000
mean,1.844367,1.871116,1.661680,1.841014,1.355581,1.228111,1.591484,1.844649,1.89298
std,0.393472,0.352027,0.556211,0.399047,0.657398,0.691716,0.557704,0.374946,0.33091
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.00000
50%,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.00000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000


count    787.000000
mean       0.286143
std        0.384057
min       -1.540761
25%        0.134706
50%        0.382787
75%        0.511545
max        1.454448
Name: 2, dtype: float64

count    8.690000e+02
mean     2.759588e-15
std      1.000000e+00
min     -2.186202e+00
25%     -8.531242e-01
50%     -2.505856e-02
75%      6.202628e-01
max      4.603463e+00
Name: 2, dtype: float64

The output of the code below show that the maximum number of NaNs in a single column is very small considering we have a total of more than 800 samples, so we did not remove any columns based on their number of NaNs. Also, the maximum number of NaNs for a single sample is lower than 10% of the number of features, so we also decided to keep all samples. 

In [23]:
# NAs per column
na_col = snp_sample.isna().sum()
display(na_col.describe())

# NAs per row
na_row = snp_sample.T.isna().sum()
display(na_row.describe())

# NAs in temperature data
display(tmin_orig[2].isna().sum())

# NAs in cloud density data
display(cloud_dens_orig[2].isna().sum())

count    1000.000000
mean        1.541000
std         2.132338
min         0.000000
25%         0.000000
50%         1.000000
75%         2.000000
max         8.000000
dtype: float64

count    869.000000
mean       1.773303
std        4.567510
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max       65.000000
dtype: float64

82
0


## Running the IRF to the naive RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from irf.ensemble import wrf_reg
from irf.utils import visualize_impurity_decrease, get_prevalent_interactions
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import itertools

def get_interactions(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
    
    # Imputing beforehand technically means we will be polluting our test data with validation data
    # when doing CV, but it saves a lot on computation. As long as we don't use the final test data, 
    # we should be fine.
    imputer = KNNImputer(n_neighbors=10)
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.fit_transform(X_test)
    
    best_params = run_rf_cv(X_train, X_test, y_train, y_test)
    
    # Reusing best parameters from RF instead of doing CV again because IRF is too computationally expensive.
    irf = wrf_reg(**best_params)
    
    print("Fitting IRF...")
    irf.fit(X_train, y_train, keep_record=False)

    print(f"R2 score: {irf.score(X_test, y_test)}")
    
    visualize_interactions(irf)
    

def run_rf_cv(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor()
    params = {'n_estimators': [100, 400, 800, 1600],
              'max_features': ['sqrt', 'log2']}

    # Only 3-fold due to low number of samples
    clf = GridSearchCV(rf, params, verbose=3, cv=3)
    
    print("Running CV...")
    clf.fit(X_train, y_train)
    
    print(f"Best parameters: {clf.best_params_}")
    print(f"R2 score: {clf.best_estimator_.score(X_test, y_test)}")
    
    return clf.best_params_


def visualize_interactions(irf):
    visualize_impurity_decrease(irf, yscale='log', xscale="linear")
    
    IMPURITY_DECREASE_THRESHOLD = 0.015
    prevalence = get_prevalent_interactions(irf, impurity_decrease_threshold=IMPURITY_DECREASE_THRESHOLD)
    
    best_genes = list(sorted(filter(lambda x: len(x[0]) == 1, prevalence.items()), key=lambda x: -x[1]))
    best_pair = max(filter(lambda x: len(x[0]) == 2, prevalence.items()), key=lambda x: x[1])

    plt.rcParams['figure.figsize'] = [10, 20]
    fig, axs = plt.subplots(4, 1)

    for i, ax in enumerate(axs.reshape(-1)):
        ax.set_title(f"{snp_sample.columns[best_genes[i][0][0]]} (Prevalence: {best_genes[i][1]:.4f})")
        ax.boxplot([y[snp_sample.iloc[:, best_genes[1][0][0]] == i] for i in (0, 1, 2)], vert=True)

    plt.tight_layout()
    plt.show()

    best_pair_names = (snp_sample.columns[best_pair[0][0]], snp_sample.columns[best_pair[0][1]])
    fig, ax = plt.subplots()
    ax.set_title(f"Best pair: {best_pair_names} (Prevalence: {best_pair[1]:.4f})")
 # Subgroup of variants. 
    combos = list(itertools.product((0, 1, 2), (0, 1, 2)))
    ax.boxplot([y[(snp_sample.iloc[:, best_pair[0][0]] == i) & (snp_sample.iloc[:, best_pair[0][1]] == j)] for i, j in combos])

    ax.set_xticklabels(combos)
    plt.show()

We note that when comparing the results of the IRF and the naive RF, the IRF performs better to the naive RF.

When looking at the box plots on the x-axis, the number indicates the number of mutations for each variant. 

It can be seen in the plots that for the single variants with most importance for temperature, having one mutations makes a big difference, but one and two mutations lead to somewhat similar results. 

In [ ]:
X_temp = snp_sample[~tmin_orig[2].isna()]
y_temp = tmin_orig[2][~tmin_orig[2].isna()]

get_interactions(X_temp, y_temp)

In [ ]:
X_cloud = snp_sample
y_cloud = cloud_dens_orig[2]

get_interactions(X_cloud, y_cloud)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dc19fcf8-fcbc-4029-b9cc-903eb0e7b73d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>